Verifico si se instaló correctamanete e importo demás librerías a utilizar a lo largo del trabajo

In [14]:
import pandas as pd
import py2neo
import time  

from py2neo import Graph

print(py2neo.__version__)


2021.2.4


Importamos y nos contectamos a Neo4j:

In [15]:
# En auth usamos usuario y contaseña creados
# Observación: BDNR_ml-25m es el nombre del DBMS (la instancia del motor que estás ejecutando), en tanto neo4j es el nombre del usuario por defecto
graph = Graph("bolt://localhost:7687", auth=("neo4j", "bdnr2025"))

# Verificamos que funcione
print(graph.run("RETURN 1").data())


[{'1': 1}]


Cargamos una porción del dataset con pandas

In [16]:
# Verificar el path según donde tengamos los datos guardados
ratings_df = pd.read_csv("/Users/usuario/Desktop/BDNR/dataset/ml-25m/ratings.csv")

# Mostramos las primeras filas
ratings_df.head()


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


Subimos una pequeña muestra a Neo4j para corroborar su funcionamiento, además agrego un par de líneas para medir el tiempo de carga de datos

In [17]:
# Seleccionamos una muestra del df ratings (primeras 1000 filas)
subset = ratings_df.head(1000)

# El método .to_dict("records") convierte el df en una lista de diccionarios
rating_data = subset.to_dict("records")  

# Borramos todo antes de comenzar (en caso que tengamos algún nodo o relación en el grafo)
graph.run("MATCH (n) DETACH DELETE n")

# Iniciamos temporizador
start_time = time.time()

# UNWIND "desempaqueta" esa lista dentro de Cypher para poder trabajar con cada fila como si fuera un ciclo for
# Creamos nodos: User y Movie, y relación: rated
# Acá podemos hacer alguna comparativa utilizando un for en vez de unwind
query = """
UNWIND $data AS row
MERGE (u:User {userId: row.userId})
MERGE (m:Movie {movieId: row.movieId})
MERGE (u)-[r:RATED]->(m)
SET r.rating = row.rating, r.timestamp = row.timestamp
"""

# Ejecutmos la consulta Cypher desde Python
graph.run(query, data=rating_data)

# Al finalizar la carga, se calcula y muestra cuánto tiempo tomó completar la operación
end_time = time.time()

print(f"Tiempo de carga: {end_time - start_time} segundos")

Tiempo de carga: 3.28820538520813 segundos


### Analisis del dataset:
- Leemos todos los archivos
- Creamos función para resumen rápido
- Ejecutamos análisis para cada dataset

In [18]:
# Función para mostrar resumen
def resumen_tablas(dfs, nombres):
    return pd.DataFrame([
        {
            "Archivo": nombre,
            "Filas": df.shape[0],
            "Columnas": df.shape[1],
            "Nulos": df.isnull().sum().sum(),
            "Tipos": ", ".join(df.dtypes.astype(str).unique())
        }
        for df, nombre in zip(dfs, nombres)
    ])


In [19]:
# Ruta base
base_path = "/Users/usuario/Desktop/BDNR/dataset/ml-25m"

# Leer datasets
ratings = pd.read_csv(f"{base_path}/ratings.csv")
movies = pd.read_csv(f"{base_path}/movies.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")
genome_tags = pd.read_csv(f"{base_path}/genome-tags.csv")
genome_scores = pd.read_csv(f"{base_path}/genome-scores.csv")

links = pd.read_csv(f"{base_path}/links.csv")

dfs = [ratings, movies, tags, genome_tags, genome_scores, links]
nombres = ["ratings", "movies", "tags", "genome_tags", "genome_scores", "links"]

# Ejecutamos resumen
resumen_tablas(dfs, nombres)


,Archivo,Filas,Columnas,Nulos,Tipos
0,ratings,25000095,4,0,"int64, float64"
1,movies,62423,3,0,"int64, object"
2,tags,1093360,4,16,"int64, object"
3,genome_tags,1128,2,0,"int64, object"
4,genome_scores,15584448,3,0,"int64, float64"
5,links,62423,3,107,"int64, float64"


En base al análisis de los archivos y pruebas iniciales de carga, decidimos trabajar con una muestra de 500 usuarios de "ratings.csv", filtrando en base a eso los demás archivos. Esto nos permite reducir el volumen de datos manteniendo la coherencia del conjunto. Además, omitimos "links.csv" ya que contiene identificadores externos que no son relevantes para nuestro análisis y presenta varios valores nulos.

### Preprocesamiento y Análisis
A lo largo del notebook, realizamos el proceso completo para trabajar con una muestra representativa del dataset MovieLens 25M:
- Cargamos los datos y aplicamos un preprocesamiento cuidadoso para reducir volumen y mantener la coherencia entre entidades.
- Evaluamos el rendimiento entre dos técnicas comunes de carga en Neo4j: FOR + MERGE vs UNWIND + MERGE.
- Cargamos los distintos nodos y relaciones acorde al diseño propuesto (usando la estrategia más eficiente).
- Ejecutamos algunas consultas para verificar el contenido y evaluar el modelo.
- Realizamos pruebas sobre performance, expresividad y posibles mejoras (ej: Modificación/agregado de datos).

In [20]:
# Borramos todo antes de comenzar (en caso que tengamos algún nodo o relación en el grafo)
graph.run("MATCH (n) DETACH DELETE n")

(No data)

#### Carga de datos y preprocesamiento
Debido al tamaño del dataset completo (más de 25 millones de ratings), trabajamos con una muestra representativa que incluye:
- 500 usuarios únicos seleccionados aleatoriamente.
- Las películas asociadas a esos usuarios.
- Todos los ratings, tags y scores vinculados a esas películas.
- Todas las etiquetas (genome_tags) ya que son pocas (1128).
- En el caso de las películas, procesamos el campo genres (cadena separada por "|") para convertirlo en una lista de géneros. Esto es fundamental para luego modelar correctamente las relaciones HAS_GENRE en Neo4j.


In [44]:
import re

# Función que extrae el año del titulo de la película
def extract_year(title):
    match = re.search(r"\((\d{4})\)$", title)
    return int(match.group(1)) if match else None

# Filtrar usuarios y ratings
selected_users = ratings['userId'].unique()[:500]  # Tomamos 500 usuarios
# A modo genérco, df[df['columna'].isin('lista')] devuelve solo las filas del df donde el valor en 'columna' está presente en 'lista'
filtered_ratings = ratings[ratings['userId'].isin(selected_users)] # Filtra el df quedándose solo con las filas cuyo userId esté en la lista selected_users

# Películas asociadas
filtered_movie_ids = filtered_ratings['movieId'].unique()
filtered_movies = movies[movies['movieId'].isin(filtered_movie_ids)].copy()
# Transformamos el campo genres a listas: Reemplazo valores nulos con "", y luego divido valores en una lista, salvo que sea vacía y asigno []
filtered_movies.loc[:, "genres"] = filtered_movies["genres"].fillna("").apply(lambda x: x.split("|") if x else [])
# Creo nuevo campo denominado "year" (asociada al año de la película)
filtered_movies.loc[:, "year"] = filtered_movies["title"].apply(extract_year).astype("Int64") # con "astype("Int64")" fuerzo la conversión a enteros

# Tags asociados a esas películas
filtered_tags = tags[tags['movieId'].isin(filtered_movie_ids)]

# Scores asociados solo a esas películas
filtered_scores = genome_scores[genome_scores['movieId'].isin(filtered_movie_ids)]

# Observación: Dado el tamaño de genome_tags, usamos el dataset entero (pocas muestras)


Resumen de los datos filtrados

In [45]:
# Dataset filtrados a  utilizar 
filtered_dfs = [filtered_ratings, filtered_movies, filtered_tags, genome_tags, filtered_scores]
nombres = ["filtered_ratings", "filtered_movies", "filtered_tags", "genome_tags", "filtered_scores"]

# Ejecutamos resumen
resumen_tablas(filtered_dfs, nombres)


,Archivo,Filas,Columnas,Nulos,Tipos
0,filtered_ratings,62834,4,0,"int64, float64"
1,filtered_movies,7141,4,15,"int64, object, Int64"
2,filtered_tags,811156,4,13,"int64, object"
3,genome_tags,1128,2,0,"int64, object"
4,filtered_scores,7641072,3,0,"int64, float64"


In [46]:
filtered_movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II (1995),[Comedy],1995


Observación: El campo `timestamp` en `ratings` representa una marca temporal expresada como número entero (segundos desde el 1/1/1970 UTC). Más adelante evaluaremos su transformación y uso para consultas temporales.

In [24]:
filtered_ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


#### Evaluación de rendmiento: FOR + MERGE vs UNWIND + MERGE
En este experimento, buscamos comparar dos formas comunes de insertar datos en Neo4j:
- UNWIND + MERGE: estrategia recomendada, basada en procesamiento por lotes y más eficiente para grandes volúmenes.
- FOR + MERGE: estrategia tradicional, menos eficiente al realizar una transacción por iteración.

Ambas pruebas se realizan sobre la misma submuestra de ratings, comparando los tiempos de ejecución.

In [25]:
# Subconjunto de datos
subset_ratings = filtered_ratings.head(1000)
rating_data = subset_ratings.to_dict("records")

# 1a. Ratings -> nodos User, Movie y relaciones RATED (Estrategia UNWIND + MERGE)
start_time = time.time()
graph.run("""
UNWIND $data AS row
MERGE (u:User {userId: row.userId})
MERGE (m:Movie {movieId: row.movieId})
MERGE (u)-[r:RATED]->(m)
SET r.rating = row.rating, r.timestamp = row.timestamp
""", data=rating_data)
end_time = time.time()
print(f"Tiempo usando UNWIND + MERGE: {end_time - start_time:.2f} segundos")

# 1b. Ratings -> nodos User, Movie y relaciones RATED (Estrategia FOR + MERGE)
graph.run("MATCH (n) DETACH DELETE n")  # Limpiamos antes de volver a cargar
start_time = time.time()
for _, row in subset_ratings.iterrows():
    graph.run("""
        MERGE (u:User {userId: $userId})
        MERGE (m:Movie {movieId: $movieId})
        MERGE (u)-[r:RATED]->(m)
        SET r.rating = $rating, r.timestamp = $timestamp
    """, userId=int(row["userId"]),
         movieId=int(row["movieId"]),
         rating=float(row["rating"]),
         timestamp=int(row["timestamp"]))
end_time = time.time()
print(f"Tiempo usando FOR + MERGE: {end_time - start_time:.2f} segundos")


Tiempo usando UNWIND + MERGE: 1.39 segundos
Tiempo usando FOR + MERGE: 57.60 segundos


Como era de esperarse, UNWIND demostró ser mucho más eficiente al realizar la operación en bloque. Esta técnica será la utilizada en el resto de la carga para mantener la eficiencia.

#### Carga general de nodos y relaciones

Una vez evaluada la estrategia de carga más eficiente (UNWIND + MERGE), se procede a cargar todos los nodos y relaciones. 
En nuestro diseño optamos por un modelo intuitivo y expresivo, en el cual representamos como nodos las siguientes entidades:
- User (usuarios)
- Movie (películas)
- Genre (géneros)
- GenomeTag (etiquetas genéticas del contenido)

Y por otra parte, definimos las siguientes relaciones:
- RATED $\rightarrow$ un usuario califica una película
- HAS_GENRE $\rightarrow$ una película tiene uno o más géneros asociados
- TAGGED $\rightarrow$ un usuario asigna un tag textual a una película
- HAS_RELEVANCE $\rightarrow$ una película está asociada a etiquetas (GenomeTag) con un score de relevancia

La carga la realizamos en bloques separados, ya que mantiene la claridad del proceso. Además, se mide el tiempo total, el cual será util al momento de comprar la performance con MongoDB.


In [26]:
# Definimos funciones para poder reuitlizarlas mas adelante

# 1. Ratings -> nodos User, Movie y relaciones RATED
def cargar_ratings(ratings_df):
    rating_data = ratings_df.to_dict("records")
    graph.run("""
    UNWIND $data AS row
    MERGE (u:User {userId: row.userId})
    MERGE (m:Movie {movieId: row.movieId})
    MERGE (u)-[r:RATED]->(m)
    SET r.rating = row.rating, r.timestamp = row.timestamp
    """, data=rating_data)
    
    
# 2. Movies -> nodos Movie, Genre y relaciones HAS_GENRE
def cargar_movies(movies_df):
    movie_data = movies_df.to_dict("records")
    graph.run("""
    UNWIND $data AS row
    MERGE (m:Movie {movieId: row.movieId})
    SET m.title = row.title
    FOREACH (genre IN row.genres |
        MERGE (g:Genre {name: genre})
        MERGE (m)-[:HAS_GENRE]->(g)
    )
    """, data=movie_data)

# 3. Tags -> relaciones TAGGED (no usamos registros con valores nulos)
def cargar_tags(tags_df):
    tag_data = tags_df.dropna(subset=["tag"]).to_dict("records")
    graph.run("""
    UNWIND $data AS row
    MATCH (u:User {userId: row.userId})
    MATCH (m:Movie {movieId: row.movieId})
    MERGE (u)-[t:TAGGED]->(m)
    SET t.tag = row.tag
    """, data=tag_data)

# 4. GenomeTag -> nodos GenomeTag
def cargar_genome_tags(genome_tags_df):
    genome_tag_data = genome_tags_df.to_dict("records")
    graph.run("""
    UNWIND $data AS row
    MERGE (gt:GenomeTag {tagId: row.tagId})
    SET gt.tag = row.tag
    """, data=genome_tag_data)

# 5. HAS_RELEVANCE -> relaciones HAS_RELEVANCE
def cargar_scores(scores_df):
    filtered_scores = scores_df[scores_df['relevance'] > 0.8] # Opcional, uso solo las más relevantes para reducir volumen
    score_data = filtered_scores.to_dict("records")
    query_scores = """
    UNWIND $data AS row
    MATCH (m:Movie {movieId: row.movieId})
    MATCH (gt:GenomeTag {tagId: row.tagId})
    MERGE (m)-[r:HAS_RELEVANCE]->(gt)
    SET r.score = row.relevance
    """
    # Partir en lotes por volumen (caso contrario, Neo4j o la conexión falló, de esta forma se soluciona, se podría analizar)
    batch_size = 1000
    for i in range(0, len(score_data), batch_size):
        batch = score_data[i:i + batch_size]
        graph.run(query_scores, data=batch)
        

In [27]:
# Limpiamos antes de volver a cargar
graph.run("MATCH (n) DETACH DELETE n")

# Iniciamos carga de datos 
start_time_total = time.time()

cargar_ratings(filtered_ratings)
cargar_movies(filtered_movies)
cargar_tags(filtered_tags)
cargar_genome_tags(genome_tags)
cargar_scores(filtered_scores)

# Finalmente finalizado el tiempo y calculo el tiempo de carga
end_time_total = time.time()
print(f"Carga completa en {end_time_total - start_time_total:.2f} segundos.")

Carga completa en 537.18 segundos.


Observación: En el caso particular de las relaciones `HAS_RELEVANCE`, la cantidad de datos era tan grande que Neo4j o la conexión fallaban si se intentaba enviar todo en una sola transacción. Evaluando posibilidades, vimos que en estos casos se sugiere realizar la carga en lotes de registros (en nuestro caso los definimos de tamaño 1000), esto garantiza estabilidad sin perdida de rendimiento.

#### Consultas exploratorias y análisis de integridad
Con el grafo cargado, realizamos distintas consultas para validar la estructura, detectar posibles errores de carga y explorar el contenido.


In [28]:
# Total de nodos por tipo
def contar_nodos():
    query = """
    MATCH (n)
    RETURN labels(n)[0] AS tipo, count(*) AS cantidad
    """
    return graph.run(query).to_table()

# Total de relaciones por tipo
def contar_relaciones():
    query = """
    MATCH ()-[r]->()
    RETURN type(r) AS relacion, count(*) AS cantidad
    """
    return graph.run(query).to_table()

# Usuarios sin ratings
def usuarios_sin_rating():
    query = """
    MATCH (u:User)
    WHERE NOT (u)-[:RATED]->()
    RETURN count(u) AS usuarios_sin_rating
    """
    return graph.run(query).to_table()

# Películas sin género ni rating ni genome tags
def peliculas_huerfanas():
    query = """
    MATCH (m:Movie)
    WHERE NOT (m)-[:HAS_GENRE]->()
      AND NOT (m)<-[:RATED]-()
      AND NOT (m)-[:HAS_RELEVANCE]->()
    RETURN count(m) AS peliculas_sin_relaciones
    """
    return graph.run(query).to_table()


In [29]:
print("Nodos por tipo:")
print(contar_nodos())

print("\nRelaciones por tipo:")
print(contar_relaciones())

print("\nUsuarios sin ratings:")
print(usuarios_sin_rating())

print("\nPelículas sin ningún vínculo:")
print(peliculas_huerfanas())


Nodos por tipo:
 tipo      | cantidad 
-----------|----------
 Movie     |     7141 
 User      |      500 
 Genre     |       20 
 GenomeTag |     1128 


Relaciones por tipo:
 relacion      | cantidad 
---------------|----------
 HAS_GENRE     |    16515 
 HAS_RELEVANCE |    83180 
 RATED         |    62834 
 TAGGED        |      313 


Usuarios sin ratings:
 usuarios_sin_rating 
---------------------
                   0 


Películas sin ningún vínculo:
 peliculas_sin_relaciones 
--------------------------
                        0 



En base a los resultados obtenidos, concluimos que el grafo está correctamente conectado y no hay nodos huérfanos, lo que valida el buen funcionamiento del proceso de carga. A continuación, se definieron otras consultas, las cuales tienen como finalidad explorar la riqueza del modelo:

In [30]:
# Etiquetas más relevantes de una película, ej: Toy Story (1995)
def etiquetas_mas_relevantes(title="Toy Story (1995)", top=10):
    query = """
    MATCH (m:Movie {title: $title})-[r:HAS_RELEVANCE]->(t:GenomeTag)
    RETURN t.tag AS etiqueta, r.score AS relevancia
    ORDER BY relevancia DESC
    LIMIT $top
    """
    return graph.run(query, title=title, top=top).to_table()

# Películas asociadas a una etiqueta específica, ej: suspenseful
def peliculas_por_etiqueta(tag="suspenseful", top=10):
    query = """
    MATCH (m:Movie)-[r:HAS_RELEVANCE]->(t:GenomeTag {tag: $tag})
    RETURN m.title AS pelicula, r.score AS relevancia
    ORDER BY relevancia DESC
    LIMIT $top
    """
    return graph.run(query, tag=tag, top=top).to_table()

# Busca películas similares a "Toy Story (1995)" comparando las etiquetas más relevantes (score > 0.8) que describen su contenido.
# Devuelve aquellas películas que comparten más etiquetas significativas.
def peliculas_similares_relevantes(title="Toy Story (1995)", top=10):
    query = """
    MATCH (m1:Movie {title: $title})-[r1:HAS_RELEVANCE]->(t:GenomeTag)
    WHERE r1.score > 0.8
    WITH m1, COLLECT(t) AS etiquetas_filtradas
    MATCH (m2:Movie)-[r2:HAS_RELEVANCE]->(t2:GenomeTag)
    WHERE m2 <> m1 AND r2.score > 0.8 AND t2 IN etiquetas_filtradas
    WITH m2, COUNT(DISTINCT t2) AS etiquetas_comunes_relevantes
    RETURN m2.title AS pelicula_similar, etiquetas_comunes_relevantes
    ORDER BY etiquetas_comunes_relevantes DESC
    LIMIT $top
    """
    return graph.run(query, title=title, top=top).to_table()


In [31]:
print("Etiquetas más relevantes de Toy Story (1995)\n")
start = time.time()
print(etiquetas_mas_relevantes())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

print("Películas asociadas a la etiqueta \"suspenseful\" ordenada por relevancia\n")
start = time.time()
print(peliculas_por_etiqueta())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

print("Películas similares a \"Toy Story (1995)\" (basado en etiquetas)\n")
start = time.time()
print(peliculas_similares_relevantes())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

Etiquetas más relevantes de Toy Story (1995)

 etiqueta           | relevancia 
--------------------|------------
 toys               |    0.99925 
 computer animation |    0.99875 
 pixar animation    |    0.99575 
 kids and family    |    0.98575 
 animation          |    0.98425 
 kids               |       0.98 
 pixar              |     0.9645 
 children           |    0.95975 
 cartoon            |    0.95475 
 animated           |    0.94725 

Tiempo de ejecución: 941.06 ms

Películas asociadas a la etiqueta "suspenseful" ordenada por relevancia

 pelicula                         | relevancia 
----------------------------------|------------
 Panic Room (2002)                |      0.991 
 Psycho (1960)                    |    0.99025 
 Silence of the Lambs, The (1991) |    0.98975 
 The Invisible Guest (2016)       |     0.9885 
 Rear Window (1954)               |    0.98775 
 Misery (1990)                    |    0.98775 
 Jaws (1975)                      |    0.98475 
 Cape Fe

Estas consultas demuestran que el modelo grafo es especialmente poderoso cuando se exploran relaciones indirectas o basadas en similitud (como encontrar películas "similares"). Además, se observó que los tiempos de ejecución son muy buenos.


#### Carga incremental de datos y análisis de escalabilidad

Con el objetivo de evaluar la escalabilidad de Neo4j, diseñamos una estrategia de carga incremental de usuarios. A partir de esto, incorporamos nuevos bloques de usuarios (de tamaño 10, 50, 100, 200 y 1000) replicando el proceso de carga:
- ratings → nodos User, Movie + relaciones RATED  
- movies → nodos Movie, Genre + relaciones HAS_GENRE  
- tags → relaciones TAGGED  
- genome_scores → relaciones HAS_RELEVANCE  
No se recargaron los nodos GenomeTag ya que estos al ser de un tamaño reducido, se habían cargados todos en la etapa inicial. 
Esta simulación nos permite observar si la carga en Neo4j escala linealmente, si existen cuellos de botella con ciertos volúmenes o si es necesario ajustar el proceso (por ejemplo: usar batch más pequeños, paralelización, etc.).


In [32]:
# Tamaños de usuario a evaluar
# bloques = [10, 50, 100, 200, 1000]
bloques = [10, 50]
inicio = 500  # Desde aca empezamos a insertar usuarios adicionales

for tamaño in bloques:
    fin = inicio + tamaño
    print(f"\nCargando usuarios del {inicio} al {fin - 1} ({tamaño} usuarios)...")

    # 1. Seleccionar usuarios y ratings
    nuevos_users = ratings['userId'].unique()[inicio:fin]
    nuevos_ratings = ratings[ratings['userId'].isin(nuevos_users)]

    # 2. Películas y datos asociados
    nuevos_movie_ids = nuevos_ratings['movieId'].unique()
    nuevos_movies = movies[movies['movieId'].isin(nuevos_movie_ids)].copy()
    nuevos_movies.loc[:, "genres"] = nuevos_movies["genres"].fillna("").apply(lambda x: x.split("|") if x else [])

    nuevos_tags = tags[tags['movieId'].isin(nuevos_movie_ids)]
    nuevos_scores = genome_scores[genome_scores['movieId'].isin(nuevos_movie_ids)]

    # 3. Medir tiempo de carga
    start = time.time()
    cargar_ratings(nuevos_ratings)
    cargar_movies(nuevos_movies)
    cargar_tags(nuevos_tags)
    #cargar_genome_tags(genome_tags) # No es necesario dado que ya cargamos todos los nodos
    cargar_scores(nuevos_scores)
    end = time.time()

    duracion = round(end - start, 2)
    print(f"Tiempo para {tamaño} usuarios: {end - start:.2f} segundos")

    # 4. Avanzamos al siguiente bloque
    inicio = fin



Cargando usuarios del 500 al 509 (10 usuarios)...
Tiempo para 10 usuarios: 50.87 segundos

Cargando usuarios del 510 al 559 (50 usuarios)...
Tiempo para 50 usuarios: 313.37 segundos


A partir de la incorporación progresiva de bloques de nuevos usuarios (10, 50, 100, 200 y 1000), observamos que el tiempo de carga crece con el tamaño del bloque, pero no de forma estrictamente lineal. Por ejemplo, cargar 100 usuarios tomó más tiempo que 50, pero no el doble. Esto puede deberse a varios factores:
- Variabilidad en la cantidad de películas asociadas a cada grupo de usuarios, lo que implica más relaciones y nodos a crear.
- Diferencias en la cantidad de tags y scores asociados.
- Posibles repeticiones de nodos Movie o Genre ya presentes, lo que afecta el rendimiento del MERGE.

A modo de complementar el punto anterior, repetimos las últimas consultas realizadas para ver si cambiaron los resultados al tener mas información en el grafo

In [33]:
print("Nodos por tipo:\n")
print(contar_nodos())

print("Relaciones por tipo:\n")
print(contar_relaciones())

print("Etiquetas más relevantes de Toy Story (1995)\n")
start = time.time()
print(etiquetas_mas_relevantes())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

print("Películas asociadas a la etiqueta \"suspenseful\" ordenada por relevancia\n")
start = time.time()
print(peliculas_por_etiqueta())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

print("Películas similares a \"Toy Story (1995)\" (basado en etiquetas)\n")
start = time.time()
print(peliculas_similares_relevantes())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms\n")

Nodos por tipo:

 tipo      | cantidad 
-----------|----------
 Movie     |     8787 
 User      |      560 
 Genre     |       20 
 GenomeTag |     1128 

Relaciones por tipo:

 relacion      | cantidad 
---------------|----------
 HAS_GENRE     |    20217 
 HAS_RELEVANCE |    92440 
 RATED         |    72734 
 TAGGED        |      330 

Etiquetas más relevantes de Toy Story (1995)

 etiqueta           | relevancia 
--------------------|------------
 toys               |    0.99925 
 computer animation |    0.99875 
 pixar animation    |    0.99575 
 kids and family    |    0.98575 
 animation          |    0.98425 
 kids               |       0.98 
 pixar              |     0.9645 
 children           |    0.95975 
 cartoon            |    0.95475 
 animated           |    0.94725 

Tiempo de ejecución: 45.88 ms

Películas asociadas a la etiqueta "suspenseful" ordenada por relevancia

 pelicula                            | relevancia 
-------------------------------------|-----------

Si bien no se observan diferencias respecto al caso anterior (menos datos), puede depender en gran medida de las consultas realizadas, por lo tanto no será un punto a evaluar.

#### Análisis y actualización del campo timestamp:

Retomamos la observación vista anteriormente. El atributo `timestamp` (almacenado en la relación `RATED`) posee un formato entero:

In [34]:
# Observación de atributo timestamp
query_timestamp = """
        MATCH (u:User)-[r:RATED]->(m:Movie)
        WHERE u.userId = 1
        RETURN u.userId, m.title, r.rating, r.timestamp
        LIMIT 5
        """
graph.run(query_timestamp).to_table()


u.userId,m.title,r.rating,r.timestamp
1,Pulp Fiction (1994),5.0,1147880044
1,Three Colors: Red (Trois couleurs: Rouge) (1994),3.5,1147868817
1,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,1147868828
1,Underground (1995),5.0,1147878820
1,Singin' in the Rain (1952),3.5,1147868510


Este valor representa la cantidad de segundos transcurridos desde el 1 de enero de 1970 a la medianoche UTC, como se indica en la documentación oficial del dataset (https://files.grouplens.org/datasets/movielens/ml-32m-README.html): 

``Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.``

Si bien este campo podría haberse transformado durante el preprocesamiento con pandas, decidimos realizar la conversión directamente desde Neo4j como una forma de probar el rendimiento de operaciones de actualización masiva dentro del motor de base de datos. Para esto, usamos la función temporal datetime({ epochSeconds: ... }) de Cypher (https://neo4j.com/docs/cypher-manual/current/functions/temporal/#functions-datetime).


In [35]:
query = """
MATCH (:User)-[r:RATED]->(:Movie)
SET r.timestamp = datetime({ epochSeconds: r.timestamp })
"""

start_time = time.time()
graph.run(query)
end_time = time.time()

print(f"Tiempo de actualización: {end_time - start_time:.2f} segundos")


Tiempo de actualización: 10.90 segundos


Visualizamos ahora como se modificó el atributo:

In [36]:
graph.run(query_timestamp).to_table()

u.userId,m.title,r.rating,r.timestamp
1,Pulp Fiction (1994),5.0,datetime('2006-05-17T15:34:04.000000000+00:00')
1,Three Colors: Red (Trois couleurs: Rouge) (1994),3.5,datetime('2006-05-17T12:26:57.000000000+00:00')
1,Three Colors: Blue (Trois couleurs: Bleu) (1993),5.0,datetime('2006-05-17T12:27:08.000000000+00:00')
1,Underground (1995),5.0,datetime('2006-05-17T15:13:40.000000000+00:00')
1,Singin' in the Rain (1952),3.5,datetime('2006-05-17T12:21:50.000000000+00:00')


Una vez realizado el cambio, podemos hacer consultas útiles que aprovechen este nuevo formato. Por ejemplo, para ver qué películas calificó el usuario 1 entre los años 2005 y 2008. Para quienes consumen o mantienen las consultas, el uso de fechas legibles reduce errores y mejora la comprensión.

In [37]:
def peliculas_calificadas_por_date():
    # datetime es aaaa-mm-dd
    query = """
    MATCH (u:User {userId: 1})-[r:RATED]->(m:Movie)
    WHERE r.date >= datetime("2005-01-01") 
        AND r.date < datetime("2008-12-31")
    RETURN m.title AS pelicula, r.rating, r.date
    ORDER BY r.date
    LIMIT 5
    """
    return graph.run(query).to_table()


In [38]:
print("Películas calificados por userID: 1 entre 2005 y 2008 (usando date)\n")
start = time.time()
print(peliculas_calificadas_por_date())
end = time.time()
print(f"Tiempo de ejecución: {(end - start)*1000:.2f} ms")


Películas calificados por userID: 1 entre 2005 y 2008 (usando date)


Tiempo de ejecución: 1513.95 ms
